In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time
import queue
import png
import cv2
from datetime import datetime
from skimage.morphology import skeletonize

%matplotlib inline
mpl.rcParams['image.cmap'] = 'bone'

# Input: DEM, ice thickness, ELA mask

In [ ]:
demPath = 'data/maps/aiguestortes/dem.png'
icePath = 'data/maps/aiguestortes/ice.png'
elaPath = 'data/maps/aiguestortes/ela.png'

# IMPORTANT: cell sizes must match those in simulation
dx = 20
dy = 20

outPath = 'data/maps/aiguestortes/'

In [ ]:
# Load DEM
Bed = cv2.imread(demPath, cv2.IMREAD_ANYDEPTH).astype(np.double)*0.1
nx, ny = Bed.shape

img = cv2.imread(icePath).astype(np.double)
Ice = img[:,:,0]/255 + img[:,:,1] + img[:,:,2]*255;
Ice = np.flipud(Ice)

elaMap = np.flipud(cv2.imread(elaPath).astype(np.int)[:,:,0])

Surf = Bed + Ice
mapShape = Bed.shape

ice_above_ELA = np.logical_and(Ice > 0, elaMap > 0)
ice_below_ELA = np.logical_and(Ice > 0, elaMap == 0)

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(131)
ax.imshow(Bed, cmap='terrain')
ax = fig.add_subplot(132)
ax.imshow(np.sqrt(Ice))
ax = fig.add_subplot(133)
ax.imshow(ice_above_ELA)

# Definitions

In [ ]:
dxy  = np.sqrt(dx*dx+dy*dy)
isq2 = 1.0/np.sqrt(2.0)
CELL_NEIGHS = [      (-1,-1), (-1,0),       (-1,1), (0,-1), (0,1),       (1,-1), (1,0),       (1,1)]
DIST_NEIGHS = [          dxy,     dx,          dxy,     dy,    dy,          dxy,    dx,         dxy]
DIR_NEIGHS  = [(-isq2,-isq2), (-1,0), (-isq2,isq2), (0,-1), (0,1), (isq2,-isq2), (1,0), (isq2,isq2)]

CELL_NEIGHS_4 = [(-1,0), (0,-1), (0,1), (1,0)]
DIST_NEIGHS_4 = [dy, dx, dx, dy]

In [ ]:
di   = np.arange(nx)
dj   = np.arange(ny)
dip  = np.hstack([np.arange(1,nx), nx-1])
dim  = np.hstack([0, np.arange(0, nx-1)])
djp  = np.hstack([np.arange(1,ny), ny-1])
djm  = np.hstack([0, np.arange(0, ny-1)])

cellIdx = (np.meshgrid(di,dj)[1], np.meshgrid(di,dj)[0])

neighs8idx = [
    (np.meshgrid(dim,djm)[1], np.meshgrid(dim,djm)[0]),
    (np.meshgrid(dim,dj)[1], np.meshgrid(dim,dj)[0]),
    (np.meshgrid(dim,djp)[1], np.meshgrid(dim,djp)[0]),
    (np.meshgrid(di,djm)[1], np.meshgrid(di,djm)[0]),
    (np.meshgrid(di,djp)[1], np.meshgrid(di,djp)[0]),
    (np.meshgrid(dip,djm)[1], np.meshgrid(dip,djm)[0]),
    (np.meshgrid(dip,dj)[1], np.meshgrid(dip,dj)[0]),
    (np.meshgrid(dip,djp)[1], np.meshgrid(dip,djp)[0])
]

In [ ]:
def gradient(field):
    grad_x = (field[dip,:] - field[dim,:])/(2*dx)
    grad_y = (field[:,djp] - field[:,djm])/(2*dy)
    return grad_x, grad_y

In [ ]:
def lineSign(A, B, X):
    return np.sign((B[0] - A[0])*(X[1] - A[1]) - (B[1] - A[1])*(X[0] - A[0]))

def linearstep(x, mi, mx):
    return (lambda t: np.where(t < 0, 0, np.where(t <= 1, t, 1)))( (x-mi)/(mx-mi) )

def smoothstep(x, mi, mx): 
    return (lambda t: np.where(t < 0, 0, np.where(t <= 1, 3*t**2-2*t**3, 1)))( (x-mi)/(mx-mi) )

In [ ]:
def encodeFieldAsRGB(values):
    c = (np.maximum(0, (values/65536.0)) * (256.0 * 256.0 * 256.0 - 1.0)).astype(int);
    cr = np.bitwise_and(np.right_shift(c, 16), 255).astype(np.uint8);
    cg = np.bitwise_and(np.right_shift(c, 8),  255).astype(np.uint8);
    cb = np.bitwise_and(c, 255).astype(np.uint8);
    c = np.dstack([cb, cg, cr])
    return c

# 1. Glacier properties

In [ ]:
Grad_B_x, Grad_B_y = gradient(Bed)
Grad_S_x, Grad_S_y = gradient(Surf)

Gradient = np.sqrt(Grad_S_x*Grad_S_x + Grad_S_y*Grad_S_y)

FlowDir = np.dstack([-Grad_S_x, -Grad_S_y])
FlowDir[Gradient > 0] = FlowDir[Gradient > 0]/Gradient[Gradient > 0, np.newaxis]

SurfaceNormal    = np.dstack([-Grad_S_x, -Grad_S_y, np.ones(mapShape)])
SurfaceNormalMag = np.linalg.norm(SurfaceNormal, axis=2)
SurfaceNormal[SurfaceNormalMag > 0] = SurfaceNormal[SurfaceNormalMag > 0]/SurfaceNormalMag[SurfaceNormalMag > 0, np.newaxis]

IceSlopeAngle = np.zeros(mapShape) 
IceSlopeAngle[SurfaceNormalMag > 0] = 0.5*np.pi - np.arccos(Gradient[SurfaceNormalMag > 0]
                                                            /SurfaceNormalMag[SurfaceNormalMag > 0])

Tau = 910*9.81*Ice*Gradient

IceFlowing = Tau > 50000  # we will consider 50 kPa the stress regime when ice begins to flow plastically

Gamma_d = 7.26e-5
Gamma_s = 3.27
Uslide  = Gamma_s * pow(Ice, 2.0) * pow(Gradient, 2.0)
Udeform = Gamma_d * pow(Ice, 4.0) * pow(Gradient, 2.0)
Utotal  = Uslide + Udeform

In [ ]:
fig = plt.figure(figsize=(20, 12))

ax = fig.add_subplot(231)
ax.imshow(Gradient, cmap='gray')
ax = fig.add_subplot(232)
ax.imshow(IceSlopeAngle, cmap='gray')
ax = fig.add_subplot(233)
ax.imshow(linearstep(Tau, 0, 150000))
ax = fig.add_subplot(234)
ax.imshow(IceFlowing)
ax = fig.add_subplot(235)
ax.imshow(linearstep(Uslide, 0, 500))
ax = fig.add_subplot(236)
ax.imshow(linearstep(Udeform, 0, 500))

In [ ]:
IceNeighbors = np.zeros(mapShape)
for n in neighs8idx:
    IceNeighbors = IceNeighbors + (Ice[n] > 0).astype(np.int)

RockIceBorder = np.logical_and(Ice <= 0, IceNeighbors > 0)
ValleyWallDF  = cv2.distanceTransform((Ice > 0).astype(np.uint8), cv2.DIST_L2, cv2.DIST_MASK_PRECISE)

In [ ]:
cv2.imwrite(outPath + 'valleydist.png', np.flipud(encodeFieldAsRGB(max(dx,dy)*ValleyWallDF).astype(np.uint8)))

# 2. Glacier segmentation

## 2.1 Upflow segmentation of glacial basins

In [ ]:
# reachable cells downflow from a set of source nodes, returns also distances and source mask
def downflowCells(surface, mask, sources):
    
    downflow = np.full(mask.shape, False)
    distance = np.full(mask.shape, 0.0)
    isSource = np.full(mask.shape, False)
    
    Q = queue.PriorityQueue()
    for s in sources:
        Q.put((0, tuple(s)))
        isSource[tuple(s)] = True
        
    while not Q.empty():
        d,p = Q.get()
        
        if downflow[p]: # already visited
            continue
    
        downflow[p] = True
        distance[p] = d
        for n,dn in zip(CELL_NEIGHS, DIST_NEIGHS):
            pn = (p[0] + n[0], p[1] + n[1])
            if 0 <= pn[0] < nx and 0 <= pn[1] < ny:  
                if not downflow[pn] and mask[pn] and surface[pn] <= surface[p]:
                    Q.put((d+dn, pn))
    
    return downflow, distance, isSource


# reachable cells upflow from a set of source nodes, returns also distances and source mask
def upflowCells(surface, mask, sources):
    
    upflow   = np.full(mask.shape, False)
    distance = np.full(mask.shape, 0.0)
    isSource = np.full(mask.shape, False)
    
    Q = queue.PriorityQueue()
    for s in sources:
        Q.put((0, tuple(s)))
        isSource[tuple(s)] = True
        
    while not Q.empty():
        d,p = Q.get()
        
        if upflow[p]: # already visited
            continue
    
        upflow[p] = True
        distance[p] = d
        for n,dn in zip(CELL_NEIGHS, DIST_NEIGHS):
            pn = (p[0] + n[0], p[1] + n[1])
            if 0 <= pn[0] < nx and 0 <= pn[1] < ny:  
                if not upflow[pn] and mask[pn] and surface[pn] >= surface[p]:
                    Q.put((d+dn, pn))
    
    return upflow, distance, isSource

In [ ]:
# segment all glaciers by propagating upflow from lowest points and keeping those big enough
def upflowSegmentation(surface, mask, minGlacierCells):
    
    GlacierIds = np.zeros(mask.shape, dtype=np.int)
    GlacierMin = np.zeros((1,2), dtype=np.int)

    remaining = mask.copy()
    idGlacier = 1

    while remaining.any():

        iceIdx = np.where(remaining.flatten())[0]
        pmin = np.unravel_index(iceIdx[surface.flatten()[iceIdx].argmin()], surface.shape)
        
        glacierSeg,_,_ = upflowCells(surface, remaining, [pmin])
        remaining[glacierSeg] = False

        if (glacierSeg.sum() >= minGlacierCells):
            print('Glacier area', idGlacier, glacierSeg.sum()*dx*dy/1e6, 'km2')
            GlacierIds[glacierSeg] = idGlacier
            GlacierMin = np.vstack([GlacierMin, pmin])
            idGlacier += 1
    
    return GlacierIds, GlacierMin, idGlacier

In [ ]:
minGlacierArea = 1e6 # 1 km2
GlacierIdMap, GlacierMinPoints, numGlaciers = upflowSegmentation(Surf, Ice > 0, minGlacierArea/(dx*dy))
print('Found', numGlaciers, 'relevant glacier areas')

In [ ]:
# Visualization
_ = plt.figure(figsize=(15,15))
plt.imshow(GlacierIdMap, cmap='terrain')
plt.plot(GlacierMinPoints[1:,1], GlacierMinPoints[1:,0], 'ro', markersize=4)

## 2.2 Subglaciers segmentation using skeleton

In [ ]:
class GlacierSkeleton:
    
    def __init__(self, mask):
        
        self.mask = mask
        self.numPoints = mask.sum()
        
        self.idMap  = np.full(self.mask.shape, -1)        
        self.points = np.zeros((self.numPoints,2), dtype=np.int)
        skeletonX, skeletonY = self.mask.nonzero()
        for i in range(self.numPoints):
            sx = skeletonX[i]
            sy = skeletonY[i]
            self.points[i] = (sx, sy)
            self.idMap[(sx,sy)] = i
        
        self.property = {}
                                
    
    def avgProperty(self, name, values, smoothingRadius):
        
        if len(values.shape) > 2:
            propVals = np.zeros((self.numPoints, values.shape[2]))
        else:            
            propVals = np.zeros((self.numPoints, 1))
            
        smoothingRadiusX = int(np.round(smoothingRadius/dx))
        smoothingRadiusY = int(np.round(smoothingRadius/dy))
        
        for i in range(self.numPoints):
            nsum = 0
            sx,sy = self.points[i]
            for di in range(-smoothingRadiusX, smoothingRadiusX+1):
                for dj in range(-smoothingRadiusY, smoothingRadiusY+1):
                    px = sx+di
                    py = sy+dj
                    if self.mask[px, py]:
                        propVals[i] += values[px,py]
                        nsum += 1
            propVals[i] /= nsum
        
        self.property[name] = propVals

        
    def assignFlowDirection(self):
        
        # in order to simplify the graph into a tree, assume each node only flows to another one
        self.flowTo  = np.full((self.numPoints, ), -1)
        
        # the lowest point on the skeleton will be the endpoint, not flowing anywhere
        self.minId = self.property['S'].argmin()
        minPoint   = self.points[self.minId]
        
        # we want to compute also how far from the endpoint we are
        self.distToMin = np.full((self.numPoints, ), -1.0)
                
        # dijkstra search, based on some cost
        PQ = queue.PriorityQueue()
        PQ.put((0, 0, (minPoint[0], minPoint[1]), -1))
        while not PQ.empty():
            
            _,d,p,pfrom = PQ.get()
            pid = self.idMap[p]

            # if we have already visited this node, skip
            if self.flowTo[pid] >= 0:
                continue

            self.flowTo[pid] = pfrom
            self.distToMin[pid] = d
            
            for i,n in enumerate(CELL_NEIGHS): 

                pn  = (p[0] + n[0], p[1] + n[1])
                nid = self.idMap[pn]
                dn  = d + DIST_NEIGHS[i]

                # if skeleton neighbor
                if self.mask[pn] and nid != self.minId:
                    # cost based only on distance
                    #PQ.put((dn, dn, pn, pid))   
                    # cost based on dist^2 and inversely proportional to stress (to better follow flows)
                    PQ.put((dn*dn/self.property['Tau'][nid], dn, pn, pid))  
                    
        
    def findSubflows(self, minSubglacierLength):
        
        # we want to assign a subglacier id to each skeleton node
        self.subglacier = np.zeros((self.numPoints, ), dtype=np.int)
        
        # keep list of source nodes and where they merge to another glacier
        self.subglacierSources = [-1]
        self.subglacierUnions  = [-1]
        self.subglacierLengths = [0]
        
        # first, compute how many nodes flow to each one
        self.inflows = np.full((self.numPoints,), 0)
        for i in range(self.numPoints):
            self.inflows[i] = np.sum(self.flowTo == i)
           
        # sources are nodes with no inflow
        sources = (self.inflows == 0).nonzero()[0]
        sourceDistToGlacier = self.distToMin[sources]
        
        subglacierId = 0
        
        # while not visited sources
        while sourceDistToGlacier.max() > minSubglacierLength:
                        
            # new subglacier
            subglacierId += 1
                        
            # get farthest source
            maxId = sourceDistToGlacier.argmax()
            farthestSource = sources[maxId]
            sourceDistToGlacier[maxId] = 0
            
            # propagate "downflow" until we either reach an endpoint or an already visited subglacier
            currNode = farthestSource
            while self.flowTo[currNode] >= 0 and self.subglacier[currNode] == 0:
                self.subglacier[currNode] = subglacierId
                currNode = self.flowTo[currNode]
            if self.subglacier[currNode] == 0:
                self.subglacier[currNode] = subglacierId
                        
            # save source and union nodes
            self.subglacierSources.append(farthestSource)
            self.subglacierUnions .append(currNode)
            self.subglacierLengths.append(sourceDistToGlacier.max())
                        
            # now update all distances for remaining sources
            for i in range(sourceDistToGlacier.size):
                
                sid = sources[i]

                # already part of a glacier?
                if self.subglacier[sid] > 0:
                    continue

                # find distance to some already seen glacier
                curr = sid
                while curr >= 0 and self.subglacier[curr] == 0:
                    curr = self.flowTo[curr]
                sourceDistToGlacier[i] = self.distToMin[sid] - (self.distToMin[curr] if curr >= 0 else 0)
                
        self.numSubglaciers = subglacierId    
        
        
    def subflowsMap(self):
        smap = np.zeros(self.mask.shape)
        for i in range(self.numPoints):
            smap[tuple(self.points[i])] = self.subglacier[i]
        return smap
        
    
    def downflowPath(self, startNode, endNode):
        nodes = []
        curr = startNode
        while self.flowTo[curr] >= 0 and curr != endNode:
            nodes.append(curr)
            curr = self.flowTo[curr]
        return nodes
        
    
    # find the closest non-skeleton cell that is reachable from both branchA and branchB without intersecting the skeleton
    def findClosestInbetweenPoint(self, unionNode, nodeBranchA, nodeBranchB, surface):

        pointU = self.points[unionNode]
        pointA = self.points[nodeBranchA]
        pointB = self.points[nodeBranchB]
        
        sharedPoint = pointU
        minDist = 10
        maxSurf = surface[tuple(pointU)]
        for di in range(-3,4):
            for dj in range(-3,4):                
                pn = [pointU[0] + di, pointU[1] + dj]
                if self.idMap[tuple(pn)] >= 0:
                    continue
                distA = np.linalg.norm(pointA - pn)
                distB = np.linalg.norm(pointB - pn)
                dist = distA + distB
                if dist < minDist or (dist == minDist and surface[tuple(pn)] > maxSurf):
                    if lineSign(pointA, pointU, pn) * lineSign(pointB, pointU, pn) < 0:
                        minDist = dist
                        sharedPoint = pn
                        maxSurf = surface[tuple(pn)]
                        
        return sharedPoint



In [ ]:
# find the closest cell in goalsMap using a BFS from startPoint and a navmap
def findClosestPoint(startPoint, navmap, goalsMap, neighSchema=CELL_NEIGHS):

    visited = np.full(navmap.shape, False)
    PQ = queue.PriorityQueue()
    PQ.put((0, startPoint))
    
    while not PQ.empty():
        d,p = PQ.get()
        if goalsMap[p]:
            return p,d
        if visited[p]:
            continue            
        visited[p] = True
        for n in neighSchema:
            pn = (p[0] + n[0], p[1] + n[1])
            if navmap[pn]:
                dn = np.linalg.norm(np.array(pn) - np.array(startPoint))
                PQ.put((dn, pn))
        
    return None, -1


# find the cell with minimum valuefield using a BFS from startPoint and a navmap
def findLocalMinimum(valuefield, navmap, startPoint):
    pcurr = startPoint
    dist  = valuefield[startPoint]
    while True:
        minDist = dist
        minNeigh = pcurr
        for n in CELL_NEIGHS:
            pn = (pcurr[0] + n[0], pcurr[1] + n[1])
            if navmap[pn]:               
                d = valuefield[pn]
                if d < minDist:
                    minDist = d
                    minNeigh = pn
        if minDist < dist:
            dist = minDist
            pcurr = minNeigh
        else:
            break
    
    return pcurr, dist

In [ ]:
# subdivides the contour into A and B, and fills the glacier using a BFS starting with cost 0 at the contours
def separateSubglaciers(intersectArea, surface, pointA, pointB, weightA, weightB, idA, idB, proportionalContour):
    
    # output
    subflowmap = np.zeros(intersectArea.shape)
    
    # get contour
    _,contours,_ = cv2.findContours(intersectArea.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    sortedContours = sorted(contours, key=cv2.contourArea, reverse=True)
    contour = [(x[0][1], x[0][0]) for x in sortedContours[0]]
    
    # lowest point on contour
    contourMin = 0
    for i,c in enumerate(contour):
        if surface[c] < surface[contour[contourMin]]:
            contourMin = i
    
    # contour on left and right sides of glacier flow
    contourClosestA = np.linalg.norm(np.array(contour - pointA), axis=1).argmin()
    contourClosestB = np.linalg.norm(np.array(contour - pointB), axis=1).argmin()
    nodesRight = []
    for i in range(contourMin+1, contourMin + len(contour)):
        cid = i%len(contour)
        nodesRight.append(contour[cid])  
        if cid == contourClosestA:
            contourA = nodesRight
            break
        elif cid == contourClosestB:
            contourB = nodesRight
            break
            
    nodesLeft = []
    for i in range(contourMin-1, contourMin - len(contour), -1):
        cid = (i + len(contour))%len(contour)
        nodesLeft.append(contour[cid])
        if cid == contourClosestA:
            contourA = nodesLeft
            break
        elif cid == contourClosestB:
            contourB = nodesLeft
            break
            
    # contourA and contourB start at glacier endpoint and follow the glacier shape up until the skeleton. 
    # if proportional, split contours such that the subflow with largest cost dies before reaching endpoint
    if weightA > weightB:
        num = int((weightB/weightA)*len(contourA)) if proportionalContour else len(contourA)-1
        contourA = contourA[len(contourA)-1 : len(contourA)-num-1 : -1]
        contourB = contourB[::-1]
    else:
        num = int((weightA/weightB)*len(contourB)) if proportionalContour else len(contourB)-1
        contourA = contourA[::-1]
        contourB = contourB[len(contourB)-1 : len(contourB)-num-1 : -1]

    # initialize expansion from contours
    PQ = queue.PriorityQueue()
    for c in contourA:
        dA = np.linalg.norm(np.array([(c[0] - pointA[0])*dx, (c[1] - pointA[1])*dy]))
        PQ.put((0, tuple(c), idA, weightA))
    for c in contourB:
        dB = np.linalg.norm(np.array([(c[0] - pointB[0])*dx, (c[1] - pointB[1])*dy]))
        PQ.put((0, tuple(c), idB, weightB))
        
    # each glacier expands from contours to neighbors, using a weighted expansion
    while not PQ.empty():
        d,p,g,w = PQ.get()
        if subflowmap[p] > 0:
            continue
        subflowmap[p] = g
        for n,dn in zip(CELL_NEIGHS, DIST_NEIGHS):
            pn = (p[0] + n[0], p[1] + n[1])
            if intersectArea[pn] and subflowmap[pn] <= 0:
                PQ.put((d + dn*w, pn, g, w))
    
    return subflowmap, contourA, contourB

In [ ]:
def computeSubglaciers(glacierMask, glacierFlowMask, glacierSurface, skeleton, 
                       maxSubglaciers, minSubflowWidth, proportionalContour):

    # output
    subglaciersMap = np.full(mapShape, -1)
    subglaciersMap[glacierMask] = 0
    subglaciersCount = 0
    
    # debug
    debugInfo = []

    # which union nodes will create a subflow?
    unionsQueue = queue.PriorityQueue()
    for ui,unionNode in enumerate(skeleton.subglacierUnions):
        
        # skip main glacier
        if unionNode < 0:
            continue
        
        # merging nodes info
        unionPoint = skeleton.points[unionNode]
        flowingNodes = [i for i in (skeleton.flowTo == unionNode).nonzero()[0] if i > 0]
        flowingGlaciers = skeleton.subglacier[flowingNodes]
        
        # we are interested in "flat" valley glaciers merging
        if skeleton.property['Gradient'][unionNode] > 0.5:
            continue
        
        # we need at least two nodes
        if len(flowingNodes) < 2: 
            continue
            
        # more than two subglaciers? keep two longest
        elif len(flowingNodes) > 2:
            lengthSort = [(skeleton.subglacierLengths[gid], nid) for nid,gid in zip(flowingNodes,flowingGlaciers)]
            lengthSort = sorted(lengthSort)            
            flowingNodes    = [lengthSort[-1][1], lengthSort[-2][1]]
            flowingGlaciers = skeleton.subglacier[flowingNodes]
 
        # we now have two glaciers, choose shortest (by skeleton construction, smallest id)
        glacierA = flowingGlaciers.max()
        unionA   = flowingNodes[flowingGlaciers.argmax()]
        sourceA  = skeleton.subglacierSources[glacierA]
        
        # add union to queue based on surface elevation      
        unionsQueue.put((float(skeleton.property['S'][unionNode]), unionNode, flowingNodes))
                
                        
    # let's compute subflows
    inglacierDF = cv2.distanceTransform(glacierMask.astype(np.uint8), cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
    skelSubflowsMap = skeleton.subflowsMap()
    currGlacierFlows = [skeleton.subglacier[skeleton.minId]]

    while not unionsQueue.empty() and subglaciersCount < maxSubglaciers:

        _,unionNode,flowingNodes = unionsQueue.get()
        flowingGlaciers = skeleton.subglacier[flowingNodes]
        unionPoint = skeleton.points[unionNode]

         # lower ids are longer glaciers, take it as main body B with affluent A
        glacierA = flowingGlaciers.max()
        glacierB = flowingGlaciers.min()
        unionA = flowingNodes[flowingGlaciers.argmax()]
        unionB = flowingNodes[flowingGlaciers.argmin()]

        # sources
        sourceA = skeleton.subglacierSources[glacierA]
        sourceB = skeleton.subglacierSources[glacierB]

        # we will always merge into already processed glaciers
        if glacierB in currGlacierFlows:

            # get the path of each branch from its source to union
            nodesA = skeleton.downflowPath(sourceA, unionNode)
            nodesB = skeleton.downflowPath(sourceB, unionNode)

            # find the starting point for the rock wall search
            pInbetween = skeleton.findClosestInbetweenPoint(unionNode, unionA, unionB, glacierSurface)
            rockSearchMap = np.logical_and(skelSubflowsMap != glacierA, skelSubflowsMap != glacierB)
            pRock, _ = findClosestPoint(tuple(pInbetween), rockSearchMap, RockIceBorder, CELL_NEIGHS_4)
            if not pRock:
                print('  Skipping union at node', unionNode, unionPoint)
                continue

            # skeleton distances
            skeldistA = cv2.distanceTransform((skelSubflowsMap != glacierA).astype(np.uint8), 
                                              cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
            skeldistB = cv2.distanceTransform((skelSubflowsMap != glacierB).astype(np.uint8), 
                                              cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
            skelsdist = np.minimum(skeldistA, skeldistB)

            # find closest rock points to each skeleton A and skeleton B
            prockA, _ = findLocalMinimum(skeldistA, RockIceBorder, pRock)
            prockB, _ = findLocalMinimum(skeldistB, RockIceBorder, pRock)

            # find closest skeleton point from each rock
            drocksSkelA = np.linalg.norm(skeleton.points[nodesA] - prockA, axis=1)
            idxskelA    = drocksSkelA.argmin()
            pskelA      = skeleton.points[nodesA[idxskelA]]

            drocksSkelB = np.linalg.norm(skeleton.points[nodesB] - prockB, axis=1)
            idxskelB    = drocksSkelB.argmin()
            pskelB      = skeleton.points[nodesB[idxskelB]]

            # refine closest skeleton point upflow until we find a width minima (as approximated by DF)
            while idxskelA > 0 and inglacierDF[tuple(skeleton.points[nodesA[idxskelA-1]])] <= \
                                   inglacierDF[tuple(skeleton.points[nodesA[idxskelA]])]:
                idxskelA -= 1
            while idxskelB > 0 and inglacierDF[tuple(skeleton.points[nodesB[idxskelB-1]])] <= \
                                   inglacierDF[tuple(skeleton.points[nodesB[idxskelB]])]:
                idxskelB -= 1
            pskelA = skeleton.points[nodesA[idxskelA]]
            pskelB = skeleton.points[nodesB[idxskelB]]
            
            flowWidthA = skeleton.property['ValleyWidth'][nodesA[idxskelA]]#drocksSkelA.min()            
            flowWidthB = skeleton.property['ValleyWidth'][nodesB[idxskelB]]#drocksSkelB.min()
            
            # skip if very small union
            if np.minimum(flowWidthA, flowWidthB)*dx < minSubflowWidth:
                continue

            # find each subglacier mask
            maskAvailable = np.logical_or(subglaciersMap == 0, subglaciersMap == glacierB)
            maskA, _, _ = downflowCells(glacierSurface, maskAvailable, skeleton.points[nodesA[idxskelA:]])
            maskB, _, _ = downflowCells(glacierSurface, maskAvailable, skeleton.points[nodesB[idxskelB:]])

            # area we are interested in separating
            maskUnion     = np.logical_or(maskA, maskB)
            maskIntersect = np.logical_and(maskA, maskB)

            # assign weight to each glacier proportional to valley width at this point
            wA = 1.0/flowWidthA
            wB = 1.0/flowWidthB
            

            # separate subglaciers
            try:
                separatedFlows, contourA, contourB = separateSubglaciers(maskUnion, glacierSurface, pskelA, pskelB, 
                                                                         wA, wB, glacierA, glacierB, proportionalContour)
            except Exception as e:
                print('  Failed at', unionNode)
                print('  ', e)
                continue

            # cells of a glacier: the ones from the intersection and the ones outside the intersection
            subglacierA = np.logical_and(maskA, np.logical_not(maskIntersect))
            subglacierA = np.logical_or(subglacierA, separatedFlows == glacierA)
            subglacierB = np.logical_and(maskB, np.logical_not(maskIntersect))
            subglacierB = np.logical_or(subglacierB, separatedFlows == glacierB)

            # update map
            subglaciersMap[subglacierA] = glacierA
            subglaciersMap[subglacierB] = glacierB
            subglaciersCount += 1

            # add glacier A as a new potential receiver of tributaries
            currGlacierFlows.append(glacierA)

            # debug
            debugInfo.append({
                'nodesA': nodesA,
                'nodesB': nodesB,
                'maskA': maskA,
                'maskB': maskB,
                'glacierA': glacierA,
                'glacierB': glacierB,
                'separatedFlows': separatedFlows,
                'rockSearchMap': rockSearchMap,
                'intersect': maskIntersect,
                'union': maskUnion,
                'pUnion': unionPoint,
                'pInbetween': pInbetween,
                'pRock': pRock,
                'prockA': prockA,
                'prockB': prockB,
                'pskelA': pskelA,
                'pskelB': pskelB,
                'wA': wA,
                'wB': wB,
                'contourA': contourA,
                'contourB': contourB
            })
    
    # propagate to extend segmented glacier flows
    # important to do this at the end, otherwise we interfere with proper subflows segmentation
    subflowIds = np.unique(subglaciersMap)
    freeNodes = np.logical_and(subglaciersMap == 0, glacierFlowMask)    

    PQ = queue.PriorityQueue()
    for i in range(skeleton.numPoints):
        p = tuple(skeleton.points[i])
        g = skeleton.subglacier[i]
        if freeNodes[p] and g > 0 and g in subflowIds:
            PQ.put((0, p, g))                
    while not PQ.empty():
        d,p,g = PQ.get()
        if not freeNodes[p]:
            continue
        freeNodes[p] = False
        subglaciersMap[p] = g
        for n,dn in zip(CELL_NEIGHS, DIST_NEIGHS):
            pn = (p[0] + n[0], p[1] + n[1])
            if freeNodes[pn] and glacierSurface[pn] < glacierSurface[p] + 1:
                PQ.put((d+dn, pn, g))

                
    return subglaciersMap, debugInfo

In [ ]:
# add first empty node to match indices
glaciersSubflowMaps = [None]
debugInfos = [None]
glacierSkeletons = [None]

for gid in range(1, numGlaciers):
    
    # masks of this glacier
    g_mask     = GlacierIdMap == gid
    g_minPoint = GlacierMinPoints[gid,:]
    g_flowing  = np.logical_and(g_mask, Tau > 50000)
    g_belowELA = np.logical_and(g_mask, ice_below_ELA)
    
    # skeletonize
    g_skeletonMask = skeletonize(np.logical_or(g_flowing, g_belowELA))
    
    # compute skeleton information and convert it into a tree (assign a unique outflow cell to each node)
    g_skeleton = GlacierSkeleton(g_skeletonMask)
    g_skeleton.avgProperty('S', Surf, 100.0)
    g_skeleton.avgProperty('Tau', Tau, 100.0)
    g_skeleton.avgProperty('Gradient', Gradient, 100.0)
    g_skeleton.avgProperty('ValleyWidth', ValleyWallDF, 100.0)
    g_skeleton.assignFlowDirection()
    
    # find subglaciers
    numMaxTributaries = 10
    minTributaryLength = 3000     # aran 3km, ecrins 5km
    minTributaryWidth = 80        # m
    proportionalContours = False  # if False, all tributaries reach the same endpoint at glacier tongue
    
    g_skeleton.findSubflows(minTributaryLength)  
    subglaciers, debugInfo = computeSubglaciers(g_mask, g_belowELA, Surf, g_skeleton,  
                                                numMaxTributaries, minTributaryWidth, proportionalContours)
    glaciersSubflowMaps.append(subglaciers)
    
    # save skeleton for later reuse
    debugInfos.append(debugInfo)
    glacierSkeletons.append(g_skeleton)
    
    print('Glacier %d: %d subglaciers' % (gid, np.unique(subglaciers,return_counts=True)[0].size-1))
    
print('done!')

In [ ]:
# this map contains a unique identifier for each subflow starting from id=2, and id=1 on other ice mass
SubflowIdMap = np.zeros(mapShape, dtype=np.int)
SubflowIdMap[Ice > 0] = 1
idSubflow = 2

# this map identifies the subglacier ids within a glacier, repeated ids possible in different basins
SubglacierIdMap = np.zeros(mapShape, dtype=np.int)


for gid in range(1, numGlaciers):
    idSubglacier = 1        
    subflowIds = np.unique(glaciersSubflowMaps[gid])
    
    for s in subflowIds:
        if s <= 0:
            continue
            
        SubflowIdMap[glaciersSubflowMaps[gid] == s] = idSubflow   
        idSubflow += 1
        
        SubglacierIdMap[glaciersSubflowMaps[gid] == s] = idSubglacier
        idSubglacier += 1        
                
numGlacierSubflows = idSubflow

In [ ]:
_ = plt.figure(figsize=(15,15))
plt.imshow(SubglacierIdMap + (Ice > 0) + (ice_below_ELA), cmap='terrain')

In [ ]:
# save
OutAreasMap = np.zeros((mapShape[0], mapShape[1], 3), dtype=np.uint8)
OutAreasMap[:,:,0] = GlacierIdMap.astype(np.uint8)
OutAreasMap[:,:,1] = SubglacierIdMap.astype(np.uint8)

cv2.imwrite(outPath + 'segmentation.png', np.flipud(OutAreasMap))  # note OpenCV uses BGR

### Debug visualization

Debug code left here as it might help understand the tributary flows segmentation algorithm.

In [ ]:
debugGlacierId = 1

d_subglaciers = glaciersSubflowMaps[debugGlacierId]

i, j = np.where(d_subglaciers >= 0)
bmini = i.min()-1
bmaxi = i.max()+1
bminj = j.min()-1
bmaxj = j.max()+1

In [ ]:
# glacier skeleton
d_skel = glacierSkeletons[debugGlacierId]

skelmap = np.zeros(mapShape)
for i in range(d_skel.numPoints):
    skelmap[tuple(d_skel.points[i])] = d_skel.subglacier[i]+50 if d_skel.subglacier[i] > 0 else 5
skelmap[Ice == 0] = 2

_ = plt.figure(figsize=(20,20))
plt.imshow(skelmap[bmini:bmaxi, bminj:bmaxj], cmap='terrain')

for dinfo in debugInfos[debugGlacierId]:
    plt.plot(dinfo['pUnion'][1]-bminj, dinfo['pUnion'][0]-bmini, 'ro', markersize=8)

In [ ]:
# contour segmentation
seeOff = 100
subflowId = 1

debug = debugInfos[debugGlacierId][subflowId]
punion = debug['pUnion']
pshared = debug['pInbetween']
pRock = debug['pRock']
prockA = debug['prockA']
prockB = debug['prockB']
pskelA = debug['pskelA']
pskelB = debug['pskelB']
nodesA = debug['nodesA']
nodesB = debug['nodesB']

bmini = pshared[0] - seeOff
bmaxi = pshared[0] + seeOff*3
bminj = pshared[1] - seeOff - 20
bmaxj = pshared[1] + seeOff

_ = plt.figure(figsize=(10,10))
plt.imshow(debug['separatedFlows'][bmini:bmaxi, bminj:bmaxj] + 1*(Ice[bmini:bmaxi, bminj:bmaxj] > 0), cmap='terrain')

for ni in nodesA:
    n = d_skel.points[ni]
    if bmini <= n[0] < bmaxi and bminj <= n[1] < bmaxj:
        plt.plot(n[1] - bminj, n[0] - bmini, 'co', markersize=8)
for ni in nodesB:
    n = d_skel.points[ni]
    if bmini <= n[0] < bmaxi and bminj <= n[1] < bmaxj:
        plt.plot(n[1] - bminj, n[0] - bmini, 'bo', markersize=8)
        

plt.plot(punion[1] - bminj, punion[0] - bmini, 'r^', markersize=10)
plt.plot(pshared[1] - bminj, pshared[0] - bmini, 'ro', markersize=12)
plt.plot(pRock[1] - bminj, pRock[0] - bmini, 'go', markersize=12)
plt.plot(prockA[1] - bminj, prockA[0] - bmini, 'yo', markersize=10)
plt.plot(prockB[1] - bminj, prockB[0] - bmini, 'yo', markersize=10)
plt.plot(pskelA[1] - bminj, pskelA[0] - bmini, 'mo', markersize=12)
plt.plot(pskelB[1] - bminj, pskelB[0] - bmini, 'mo', markersize=12)

for c in debug['contourA']:
    if bmini <= c[0] < bmaxi and bminj <= c[1] < bmaxj:
        plt.plot(c[1] - bminj, c[0] - bmini, 'co', markersize=4)
for c in debug['contourB']:    
    if bmini <= c[0] < bmaxi and bminj <= c[1] < bmaxj:
        plt.plot(c[1] - bminj, c[0] - bmini, 'bo', markersize=4)

# 3. Glacier Feature Maps

## Seracs

Caused by sudden discontinuities in moving ice surface. We will define a discontinuity between a node and its neighbor if the surface of the neighbor is lower than the node's bedrock.

In [ ]:
BrokenSurfaceNeighs = np.zeros(mapShape, dtype=np.int)
for n in neighs8idx:
    alignedToFlow = np.einsum('ijk,kij->ij', FlowDir, np.array(n) - np.array(cellIdx)) > 0
    discontinuity = np.logical_and(Ice[n] > 0, Bed > Surf[n])
    BrokenSurfaceNeighs = BrokenSurfaceNeighs + np.logical_and(discontinuity, alignedToFlow).astype(np.int)
BrokenSurfaceNeighs[Ice <= 0] = 0

# Seracs: discontinuity + moving ice
Seracs = np.minimum(BrokenSurfaceNeighs/4.0, 1.0) * IceFlowing

In [ ]:
bmini = 500
bmaxi = 800
bminj = 400
bmaxj = 700

_ = plt.figure(figsize=(10,10))
plt.imshow((Seracs*3.0 + (Ice > 0))[bmini:bmaxi,bminj:bmaxj])

In [ ]:
cv2.imwrite(outPath + 'seracs.png', np.flipud((Seracs*255).astype(np.uint8)))

## Rimayes

Appear at the bottom of steep ice walls, at the interface between this steep and frozen stagnant ice and moving body of the glacier.

In [ ]:
NumIcewallNeighs = np.zeros(mapShape, dtype=np.int)
NumFlowingNeighs = np.zeros(mapShape, dtype=np.int)
NumFlatNeighs    = np.zeros(mapShape, dtype=np.int)
NumSteepNeighs   = np.zeros(mapShape, dtype=np.int)

flowAngle = np.cos(np.radians(90.0))
steepIce  = IceSlopeAngle > np.radians(30.0)
flatIce   = np.logical_not(steepIce)

for n in neighs8idx:
    flowAlignedNeigh = np.einsum('ijk,kij->ij', FlowDir, np.array(n) - np.array(cellIdx)) < flowAngle
    
    NumIcewallNeighs = NumIcewallNeighs + np.logical_and.reduce([steepIce[n], flowAlignedNeigh]).astype(np.int)    
    NumFlowingNeighs = NumFlowingNeighs + np.logical_and(np.logical_not(steepIce[n]), IceFlowing[n]).astype(np.int)
    
    NumFlatNeighs  = NumFlatNeighs + flatIce[n].astype(np.int)
    NumSteepNeighs = NumSteepNeighs + steepIce[n].astype(np.int)

Rimayes = np.logical_and.reduce([steepIce,
                                 Tau > 30000,
                                 Tau < 60000,
                                 ice_above_ELA, 
                                 NumIcewallNeighs >= 3, 
                                 NumFlatNeighs >= 2])

In [ ]:
bmini = 500
bmaxi = 800
bminj = 450
bmaxj = 750

_ = plt.figure(figsize=(10,10))
plt.imshow((Rimayes*5.0 + (Ice > 0))[bmini:bmaxi,bminj:bmaxj])

In [ ]:
cv2.imwrite(outPath + 'rimayes.png', np.flipud((Rimayes*255).astype(np.uint8)))

## Icefalls

Icefalls form when ice is flowing under high pressure, and at steep angles. 

In [ ]:
# from the pixel-based segmentation of icefalls, creates individual falls and improves their shape by extending them
# to the rock walls, as well as omitting small segmented areas not large enough to be interesting
def icefallSegmentation(icefalls, glacierMask, surface, wallDist, minIcefallSize, maxExtendDist):
    
    FallIds = np.zeros(icefalls.shape, dtype=np.int)
    FallIds[icefalls] = -1
    idIcefall = 1
    
    while (FallIds == -1).any():
        
        # segment icefall and extend toward valley borders if close enough
        Q = queue.Queue()
        Q.put((0, tuple(np.argwhere(FallIds == -1)[0])))
        while not Q.empty():
            d,p = Q.get()
            if FallIds[p] > 0 or d > maxExtendDist:
                continue
            FallIds[p] = idIcefall
            for n,dn in zip(CELL_NEIGHS, DIST_NEIGHS):
                pn = (p[0] + n[0], p[1] + n[1])
                if FallIds[pn] < 0:
                    Q.put((0,pn))
                elif FallIds[pn] == 0:
                    if wallDist[pn]*dx <= maxExtendDist and glacierMask[pn] and surface[pn] > surface[p]:
                        Q.put((d+dn,pn))
                
            
        # not large enough? do not count this icefall
        if np.logical_and(FallIds == idIcefall, icefalls).sum()*dx*dy < minIcefallSize:
            FallIds[FallIds == idIcefall] = 0
        else:
            idIcefall += 1
        
    return FallIds

In [ ]:
icefallFlow  = 100000 # Pa
icefallAngle = np.radians(15)

# icefall locations
IcefallCandidates = np.logical_and.reduce([Tau > icefallFlow, IceSlopeAngle > icefallAngle])

# clean segmentation
minIcefallSize = 0.05e6 # km2
maxExtendDist  = 100.0  # m
IcefallIds = icefallSegmentation(IcefallCandidates, Ice > 0, Surf, ValleyWallDF, minIcefallSize, maxExtendDist)
Icefalls   = IcefallIds > 0

In [ ]:
# show the comparison before and after cleanup of icefalls
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(121)
ax.imshow(Icefalls*5.0 + 1.0*(Ice > 0) + 1.0*ice_below_ELA)
ax = fig.add_subplot(122)
ax.imshow(IcefallCandidates*5.0 + 1.0*(Ice > 0) + 1.0*ice_below_ELA)

In [ ]:
cv2.imwrite(outPath + 'icefalls.png', np.flipud((Icefalls*255).astype(np.uint8)))
cv2.imwrite(outPath + 'icefallsRaw.png', np.flipud((IcefallCandidates*255).astype(np.uint8)))

## Ogives

Ogives may form at the base of an icefall and generate an ondulation pattern.

In [ ]:
flowAngle = np.cos(np.radians(45.0))
flatAngle = np.radians(15.0)

NumIcefallNeighs = np.zeros(mapShape, dtype=np.int)
for n in neighs8idx:
    flowAligned      = np.einsum('ijk,ijk->ij', FlowDir, FlowDir[n]) > flowAngle
    flowFromNeigh    = np.einsum('ijk,kij->ij', FlowDir, np.array(n) - np.array(cellIdx)) < -flowAngle
    NumIcefallNeighs = NumIcefallNeighs + np.logical_and.reduce([Icefalls[n], flowAligned, flowFromNeigh]).astype(np.int)

OgivesBorder = np.logical_and.reduce([np.logical_not(Icefalls), 
                                      SubglacierIdMap > 0,
                                      IceFlowing,
                                      IceSlopeAngle < flatAngle,
                                      ice_below_ELA, 
                                      np.logical_and(NumIcefallNeighs >= 2, NumIcefallNeighs <= 4)])

In [ ]:
def createOgivesMap(icefalls, ogivesBorder, surface, flowDir, subflowsMap):
    
    # parameters
    minOgiveArea  = 0.1e6
    minOgiveNodes = 6  # 100m at 20m/pix
    maxFlowDev    = np.radians(25)
    numSubflows   = subflowsMap.max() + 1
    
    # result
    OgiveMap = np.full(mapShape, -1, dtype=np.int)
    OgiveMap[ogivesBorder] = 0
    OgiveSources = [[]]
    AllSources = []
    OgiveDist = np.full(mapShape, -1.0)
    
    idOgive = 1
    while (OgiveMap == 0).any():
        
        # minimum point
        omapIdx = np.where(OgiveMap.flatten() == 0)[0]
        pmin = np.unravel_index(omapIdx[surface.flatten()[omapIdx].argmin()], surface.shape)

        # 1) find connected ogive source nodes
        ogiveNodes = []
        sumDprod = 0.0
        ogiveSubflow = np.zeros((numSubflows,), dtype=np.int)
        Q = queue.Queue()
        Q.put((pmin, 0))
        while not Q.empty():
            p,d = Q.get()
            if OgiveMap[p] > 0:
                continue
            OgiveMap[p] = idOgive
            ogiveNodes.append(p)
            sumDprod += d
            if subflowsMap[p] >= 0:
                ogiveSubflow[subflowsMap[p]] += 1
            for n,d in zip(CELL_NEIGHS,DIR_NEIGHS):
                pn = (p[0] + n[0], p[1] + n[1])
                if OgiveMap[pn] == 0:
                    dprod = flowDir[pn[0],pn[1],0]*d[0] + flowDir[pn[0],pn[1],1]*d[1]
                    Q.put((pn, np.abs(dprod)))
                    
        AllSources.append(ogiveNodes)          
        
        # if ogive too small or too unaligned with flow
        if len(ogiveNodes) < minOgiveNodes or sumDprod/len(ogiveNodes) > np.cos(0.5*np.pi - maxFlowDev):
            OgiveMap[OgiveMap == idOgive] = -1
            continue

            
        # 2) subflows affected by this ogive
        flowIds = np.where(ogiveSubflow > 0)[0]
        if len(flowIds) == 0:
            OgiveMap[OgiveMap == idOgive] = -1
            continue

            
        # 3) downflow from ogive nodes, stopping if another icefall is found
        distMap = np.full(OgiveDist.shape, -1.0)
        PQ = queue.PriorityQueue()
        for n in ogiveNodes:
            if subflowsMap[n] in flowIds:
                PQ.put((0, n))
        while not PQ.empty():
            d,p = PQ.get()
            if distMap[p] >= 0:
                continue                
            distMap[p] = d
            for n,dn in zip(CELL_NEIGHS, DIST_NEIGHS):
                pn = (p[0] + n[0], p[1] + n[1])
                if subflowsMap[pn] in flowIds and not icefalls[pn] and distMap[pn] < 0 and surface[pn] <= surface[p]:
                    PQ.put((d+dn, pn))
                    
        # filter out small sections
        for fid in flowIds:
            fidOgive = np.logical_and(subflowsMap == fid, distMap >= 0)
            if fidOgive.sum()*dx*dy < minOgiveArea:
                distMap[fidOgive] = -1
        
        if (distMap >= 0).sum()*dx*dy < minOgiveArea:
            OgiveMap[OgiveMap == idOgive] = -1
            continue
                    
                    
        # copy result
        OgiveMap [distMap >= 0] = idOgive
        OgiveDist[distMap >= 0] = distMap[distMap >= 0]
        OgiveSources.append(ogiveNodes)

        # go for the next one
        idOgive += 1
    
    
    return OgiveMap, OgiveDist, OgiveSources, AllSources
    
    

In [ ]:
OgivesMap, OgivesDist, OgivesSources, AllSources = createOgivesMap(Icefalls, OgivesBorder, Surf, FlowDir, SubflowIdMap)

print('Found %d ogive areas'%OgivesMap.max())

In [ ]:
OgivesCtrDist = np.zeros(OgivesMap.shape)
for i in range(1, OgivesMap.max()+1):
    ogiveSkeletonMask = np.logical_not(skeletonize(OgivesMap == i))
    dist = cv2.distanceTransform(ogiveSkeletonMask.astype(np.uint8), cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
    dist[OgivesMap != i] = 0
    OgivesCtrDist += dist
OgivesCtrDist *= np.maximum(dx, dy)

In [ ]:
bmini = 550
bmaxi = 850
bminj = 250
bmaxj = 550

fig = plt.figure(figsize=(20,10))

ax = fig.add_subplot(121)
ax.imshow((OgivesMap + 300*(OgivesMap > 0) + 3.0*SubglacierIdMap + 100*Icefalls)[bmini:bmaxi,bminj:bmaxj], cmap='terrain')
for os in AllSources:
    for s in os:
        if bmini <= s[0] <= bmaxi and bminj <= s[1] <= bmaxj:
            ax.plot(s[1]-bminj, s[0]-bmini, 'mo', markersize=3)
for os in OgivesSources:
    for s in os:
        if bmini <= s[0] <= bmaxi and bminj <= s[1] <= bmaxj:
            ax.plot(s[1]-bminj, s[0]-bmini, 'ro', markersize=4)
            

ax = fig.add_subplot(122)
ax.imshow(np.cos((OgivesDist + OgivesCtrDist)/30.0)[bmini:bmaxi,bminj:bmaxj])

In [ ]:
cv2.imwrite(outPath + 'ogivesId.png', np.flipud(np.maximum(0, OgivesMap)).astype(np.uint8))
cv2.imwrite(outPath + 'ogivesDistSrc.png', np.flipud(encodeFieldAsRGB(OgivesDist).astype(np.uint8)))
cv2.imwrite(outPath + 'ogivesDistCtr.png', np.flipud(encodeFieldAsRGB(OgivesCtrDist).astype(np.uint8)))

## Crevasses

Crevasses appear due to sudden steep changes in glacier slope, different velocities, irregularities on the bedrock or the valley widening/narrowing.

First, let's compute some properties that will be useful to place crevasses

In [ ]:
# gradient of bedrock in the direction of flow
Dflow_bed = (FlowDir[:,:,0]*Grad_B_x + FlowDir[:,:,1]*Grad_B_y)*(Ice > 0)

# derivative of stress in perpendicular direction to flow
Grad_tau_x, Grad_tau_y = gradient(Tau)
Dperp_tau = (FlowDir[:,:,1]*Grad_tau_x - FlowDir[:,:,0]*Grad_tau_y)*(ValleyWallDF > 1)

# speed derivatives along flow and perpendicular to flow
Grad_u_x, Grad_u_y = gradient(Utotal)
Dflow_speed = (FlowDir[:,:,0]*Grad_u_x + FlowDir[:,:,1]*Grad_u_y)*(ValleyWallDF > 1)
Dperp_speed = (FlowDir[:,:,1]*Grad_u_x - FlowDir[:,:,0]*Grad_u_y)*(ValleyWallDF > 1)

The distance from the glacier terminus will be used to determine some crevasse types

In [ ]:
# distance map to end of glaciers
GlacierEndDist = np.full(mapShape, -1)
GlacierMaxDist = np.full(mapShape, -1)
for i in range(1, numGlaciers):
    print('Computing distmap of glacier %d'%i, end='\r')
    gmask = GlacierIdMap == i
    _,gdist,_ = upflowCells(Surf, gmask, [(GlacierMinPoints[i,0], GlacierMinPoints[i,1])])
    GlacierEndDist[gmask] = gdist[gmask]
    GlacierMaxDist[gmask] = gdist[gmask].max()

We want to compute the width of the glacier over its surface. To do so, we will use the value at the glacier skeleton as extracted from the distance field, and propagate it to each skeleton node closest cells.

In [ ]:
sk_valleyW = np.zeros(mapShape)

for gid in range(1, numGlaciers):
    print('Computing valley width of glacier %d'%gid, end='\r')
    
    # mask and skeleton
    gmask = GlacierIdMap == gid
    skel = glacierSkeletons[gid]

    # closest skeleton node map
    skelNodeMap = np.full(mapShape, -1)
    PQ = queue.PriorityQueue()
    for i in range(skel.numPoints):
        if skel.subglacier[i] > 0:
            PQ.put((0, (skel.points[i][0], skel.points[i][1]), i))
    while not PQ.empty():
        d,p,i = PQ.get()
        if skelNodeMap[p] >= 0:
            continue
        skelNodeMap[p] = i
        for n,dn in zip(CELL_NEIGHS, DIST_NEIGHS):
            pn = (p[0] + n[0], p[1] + n[1])
            if gmask[pn] and skelNodeMap[pn] < 0:
                PQ.put((d+dn, pn, i))
    
    # valley width from skeleton
    sk_valleyW[gmask] = (skel.property['ValleyWidth'][skelNodeMap].squeeze())[gmask]
    
    # compute some derivatives along the skeleton
    Dskel_valleyW = np.zeros((skel.numPoints,))
    for i in range(skel.numPoints):
        n = skel.flowTo[i]
        d = np.linalg.norm(skel.points[i] - skel.points[n])
        if n >= 0:
            Dskel_valleyW[i] = (skel.property['ValleyWidth'][n] - skel.property['ValleyWidth'][i])/d
                                                                                
# some blurring, helps to remove the discontinuities of the skeleton closest node field
sk_valleyW = cv2.GaussianBlur(sk_valleyW, (9,9), 0)

# set to 0 outside iced areas
sk_valleyW[Ice <= 0] = 0

print('\nDone')

In [ ]:
# valley with derivative along flow direction
Grad_skValleyW_x, Grad_skValleyW_y = gradient(sk_valleyW)
Dflow_skValleyW = (FlowDir[:,:,0]*Grad_skValleyW_x + FlowDir[:,:,1]*Grad_skValleyW_y)*(ValleyWallDF > 1)

In [ ]:
cv2.imwrite(outPath + 'valleywidth.png', np.flipud(encodeFieldAsRGB(max(dx,dy)*sk_valleyW).astype(np.uint8)))

**MARGINAL CREVASSES** form due to stress on ice close to glacier margins that moves much lower than ice on glacier center.

We will place them below ELA, on relatively flat areas and where the stress is lower than deformation threshold (50 kPa).
The probability will decrease towards the center of the glacier, and will be proportional to the variation of stress towards the wall

In [ ]:
# where to find them
MarginalCrevasses = np.logical_and.reduce([ice_below_ELA,
                                           IceSlopeAngle < np.radians(10),
                                           Tau < 70000]).astype(np.float)

# intensity
MarginalCrevasses = MarginalCrevasses * np.abs(Dperp_tau)
MarginalCrevasses = (MarginalCrevasses - MarginalCrevasses.min())/(MarginalCrevasses.max() - MarginalCrevasses.min())
MarginalCrevasses = np.sqrt(MarginalCrevasses * (1 - smoothstep(ValleyWallDF, 3, 10)))

In [ ]:
i, j = np.where(GlacierIdMap == 1)
bmini = i.min()-1
bmaxi = i.max()+1
bminj = j.min()-1
bmaxj = j.max()+1

fig = plt.figure(figsize=(10,10))
plt.imshow(MarginalCrevasses[bmini:bmaxi,bminj:bmaxj])

In [ ]:
cv2.imwrite(outPath + 'crevassesMargin.png', np.flipud((MarginalCrevasses*255).astype(np.uint8)))

**LONGITUDINAL CREVASSES** form when the valley widens or bends, and there is a compressive stress in the flow direction (i.e. speed slows down) and expansive in direction perpendicular to flow.

In [ ]:
# compressive forces in the direction of flow -> speed slows down
loncrev_slowing = smoothstep(Dflow_speed, 0, -1)

# expansive forces in direction perpendicular to flow -> longitudinal openings (acceleration perpendicular to flow)
loncrev_expanse = smoothstep(np.abs(Dperp_speed), 0, 10)

# they are more usual towards the end of a glacier
loncrev_closeToEnd = smoothstep(GlacierEndDist/GlacierMaxDist, 0.4, 0.1)

# they occur due to valley opening -> Dflow_skValleyW > 0, 
# we use step from a negative derivative to avoid sharp transitions
loncrev_vOpening = smoothstep(Dflow_skValleyW, -0.01, 0)


# probability
LongitudinalCrevasses = (loncrev_slowing + loncrev_expanse) * loncrev_closeToEnd * loncrev_vOpening * ice_below_ELA
LongitudinalCrevasses = (LongitudinalCrevasses - LongitudinalCrevasses.min())/\
                        (LongitudinalCrevasses.max() - LongitudinalCrevasses.min())

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(LongitudinalCrevasses[bmini:bmaxi, bminj:bmaxj])

In [ ]:
cv2.imwrite(outPath + 'crevassesLong.png', np.flipud((LongitudinalCrevasses*255).astype(np.uint8)))

**TRANSVERSE CREVASSES** are the most common type. They form when the valley steepens and the flow speeds up, due to tensile stresses on the ice. They form across the glacier, and the margin is typically crevasse-free.

In [ ]:
# second order directional derivative of bedrock to detect bumps on the terrain
Dflow_bed_gx, Dflow_bed_gy = gradient(Dflow_bed)
D2flow_bed = FlowDir[:,:,0]*Dflow_bed_gx + FlowDir[:,:,1]*Dflow_bed_gy

In [ ]:
# ice accelerating: D speed / d flow > 0
transcrev_accel = smoothstep(Dflow_speed, 0, 1)

# bedrock irregularities
transcrev_bedbumps = np.sqrt(smoothstep(D2flow_bed, 0, -0.005))

# valley steepens
transcrev_steep = linearstep(IceSlopeAngle, np.radians(10), np.radians(30))

# distance to margin
transcrev_center = smoothstep(ValleyWallDF, 3, 10)

# map
TransverseCrevasses = np.logical_and.reduce([Tau > 50000,
                                             np.logical_not(Icefalls)])
TransverseCrevasses = TransverseCrevasses * (transcrev_accel + transcrev_bedbumps + transcrev_steep)/3.0 * transcrev_center

In [ ]:
_ = plt.figure(figsize=(10,10))
plt.imshow(TransverseCrevasses)

In [ ]:
cv2.imwrite(outPath + 'crevassesTrans.png', np.flipud((TransverseCrevasses*255).astype(np.uint8)))